In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img

In [7]:
path = "train/hat/00d94e21-5891-492e-be0e-792e7338c077.jpg"
img = load_img(path,target_size = (299,299))

In [8]:
x = np.array(img)

In [16]:
from tensorflow.keras.applications.xception import Xception,preprocess_input,decode_predictions

In [12]:
model = Xception(weights = 'imagenet',input_shape = (299,299,3))

2025-07-16 18:56:42.762358: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


91884032/91884032 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [17]:
X = np.array([x])
X = preprocess_input(X)

In [18]:
pred = model.predict(X)
decode_predictions(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 755ms/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


[[('n03424325', 'gasmask', np.float32(0.15357602)),
  ('n02769748', 'backpack', np.float32(0.08983881)),
  ('n03709823', 'mailbag', np.float32(0.06401253)),
  ('n03595614', 'jersey', np.float32(0.04093357)),
  ('n02834397', 'bib', np.float32(0.024432717))]]